In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/twitter-dataset/train.csv
/kaggle/input/test-data/test.csv


In [2]:
df = pd.read_csv('/kaggle/input/twitter-dataset/train.csv')

In [3]:
df.head(15)

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
5,28b57f3990,http://www.dothebouncy.com/smf - some shameles...,http://www.dothebouncy.com/smf - some shameles...,neutral
6,6e0c6d75b1,2am feedings for the baby are fun when he is a...,fun,positive
7,50e14c0bb8,Soooo high,Soooo high,neutral
8,e050245fbd,Both of you,Both of you,neutral
9,fc2cbefa9d,Journey!? Wow... u just became cooler. hehe....,Wow... u just became cooler.,positive


In [4]:
len(df)

27481

In [5]:
df['sentiment'].unique()

array(['neutral', 'negative', 'positive'], dtype=object)

In [6]:
df.groupby('sentiment').nunique()

,textID,text,selected_text
sentiment,,,
negative,7781,7781,5861
neutral,11118,11117,11111
positive,8582,8582,5537


In [7]:
df = df[['selected_text', 'sentiment']]
df.head()

,selected_text,sentiment
0,"I`d have responded, if I were going",neutral
1,Sooo SAD,negative
2,bullying me,negative
3,leave me alone,negative
4,"Sons of ****,",negative


In [8]:
df['selected_text'].isnull().sum()

1

In [9]:
df['selected_text'].fillna("No content", inplace = True)
print("Done")

Done


/tmp/ipykernel_33/2882932491.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['selected_text'].fillna("No content", inplace = True)


In [10]:
import re
from bs4 import BeautifulSoup
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

def preprocess_text(texts):
    preprocessed_texts = []
    stop_words = set(stopwords.words('english'))
    
    for text in texts:
        # Remove HTML tags
        text = BeautifulSoup(text, 'html.parser').get_text()

        # Remove URLs
        text = re.sub(r'https?://[A-Za-z0-9./]+', '', text)

        # Tokenize text
        tokenizer = TweetTokenizer()
        tokens = tokenizer.tokenize(text)

        # Remove emails
        tokens = [token for token in tokens if not re.match(r'\S+@\S+', token)]

        # Remove new lines characters
        tokens = [token for token in tokens if token != '\n']

        # Remove distracting single quotes
        tokens = [token.replace("'", "") for token in tokens]

        # Remove all punctuation signs and stopwords
        tokens = [token.lower() for token in tokens if token.isalnum() and token.lower() not in stop_words]

        # Detokenize text
        preprocessed_texts.append(' '.join(tokens))

    return np.array(preprocessed_texts)
# Preprocess 'tweet' column
df['selected_text'] = preprocess_text(df['selected_text'])

print(df)

df['sentiment'] = df['sentiment'].replace({'negative': 0, 'positive': 1, 'neutral': 2})

max_words = 5000
max_len = 200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['selected_text'])
sequences = tokenizer.texts_to_sequences(df['selected_text'])
X = pad_sequences(sequences, maxlen=max_len)

# Convert sentiment column to numpy array
y = np.array(df['sentiment'])

print(X)


2024-04-11 12:30:56.369636: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-11 12:30:56.369802: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-11 12:30:56.559664: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/tmp/ipykernel_33/2864154924.py:15: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, 'html.parser').get_text()


                            selected_text sentiment
0                         responded going   neutral
1                                sooo sad  negative
2                                bullying  negative
3                             leave alone  negative
4                                    sons  negative
...                                   ...       ...
27476                                lost  negative
27477                               force  negative
27478                            yay good  positive
27479                               worth  positive
27480  flirting going atg smiles yay hugs   neutral

[27481 rows x 2 columns]


/tmp/ipykernel_33/2864154924.py:45: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'] = df['sentiment'].replace({'negative': 0, 'positive': 1, 'neutral': 2})


[[   0    0    0 ...    0    0    9]
 [   0    0    0 ...    0  299   25]
 [   0    0    0 ...    0    0    0]
 ...
 [   0    0    0 ...    0  116    1]
 [   0    0    0 ...    0    0  451]
 [   0    0    0 ... 2327  116  560]]


In [11]:
print(y)

[2 0 0 ... 1 1 2]


In [12]:
from sklearn.model_selection import train_test_split

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print("Train set size:", len(X_train))
print("Test set size:", len(X_test))
print("Train labels size:", len(y_train))
print("Test labels size:", len(y_test))

Train set size: 21984
Test set size: 5497
Train labels size: 21984
Test labels size: 5497


In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.callbacks import ModelCheckpoint

# Compute class weights
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(enumerate(class_weights))

# Define your neural network model
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128))
model.add(Bidirectional(LSTM(128, dropout=0.3, recurrent_dropout=0.3, return_sequences=True)))
model.add(Bidirectional(LSTM(64, dropout=0.3, recurrent_dropout=0.3)))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

# Compile your model with a lower learning rate
optimizer = Adam(learning_rate=0.0005)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Define the filepath for saving the model
filepath = "best_model.keras"

# Define the ModelCheckpoint callback
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

# Train your model with the ModelCheckpoint callback
history = model.fit(X_train, y_train, batch_size=64, epochs=15, validation_split=0.2, class_weight=class_weights_dict, callbacks=[checkpoint])

# Evaluate your model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


Epoch 1/15
275/275 ━━━━━━━━━━━━━━━━━━━━ 0s 864ms/step - accuracy: 0.4723 - loss: 0.9948
Epoch 1: val_accuracy improved from -inf to 0.72527, saving model to best_model.keras
275/275 ━━━━━━━━━━━━━━━━━━━━ 269s 937ms/step - accuracy: 0.4727 - loss: 0.9945 - val_accuracy: 0.7253 - val_loss: 0.7086
Epoch 2/15
275/275 ━━━━━━━━━━━━━━━━━━━━ 0s 854ms/step - accuracy: 0.7291 - loss: 0.6954
Epoch 2: val_accuracy improved from 0.72527 to 0.77234, saving model to best_model.keras
275/275 ━━━━━━━━━━━━━━━━━━━━ 257s 918ms/step - accuracy: 0.7291 - loss: 0.6953 - val_accuracy: 0.7723 - val_loss: 0.5820
Epoch 3/15
275/275 ━━━━━━━━━━━━━━━━━━━━ 0s 874ms/step - accuracy: 0.8232 - loss: 0.4972
Epoch 3: val_accuracy improved from 0.77234 to 0.78485, saving model to best_model.keras
275/275 ━━━━━━━━━━━━━━━━━━━━ 268s 940ms/step - accuracy: 0.8232 - loss: 0.4972 - val_accuracy: 0.7849 - val_loss: 0.5462
Epoch 4/15
275/275 ━━━━━━━━━━━━━━━━━━━━ 0s 859ms/step - accuracy: 0.8530 - loss: 0.4119
Epoch 4: val_accuracy

In [31]:
# Function to preprocess a single text
def preprocess_single_text(text):
    stop_words = set(stopwords.words('english'))

    # Remove HTML tags
    text = BeautifulSoup(text, 'html.parser').get_text()

    # Remove URLs
    text = re.sub(r'https?://[A-Za-z0-9./]+', '', text)

    # Tokenize text
    tokenizer = TweetTokenizer()
    tokens = tokenizer.tokenize(text)

    # Remove emails
    tokens = [token for token in tokens if not re.match(r'\S+@\S+', token)]

    # Remove new lines characters
    tokens = [token for token in tokens if token != '\n']

    # Remove distracting single quotes
    tokens = [token.replace("'", "") for token in tokens]

    # Remove all punctuation signs and stopwords
    tokens = [token.lower() for token in tokens if token.isalnum() and token.lower() not in stop_words]

    # Detokenize text
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text

# Preprocess a single text
text_to_classify = "I stayed at this hotel for a night and it was neither great nor terrible. The room was clean and comfortable, but the amenities were nothing special. It served its purpose for a short stay."
preprocessed_text = preprocess_single_text(text_to_classify)

# Tokenize and pad the preprocessed text
sequence = tokenizer.texts_to_sequences([preprocessed_text])
padded_sequence = pad_sequences(sequence, maxlen=max_len)

# Predict the sentiment class
predicted_class = np.argmax(model.predict(padded_sequence), axis=-1)[0]

# Map predicted class to sentiment label
sentiment_mapping = {0: 'negative', 1: 'positive', 2: 'neutral'}
predicted_sentiment = sentiment_mapping[predicted_class]

print("Predicted Sentiment:", predicted_sentiment)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
Predicted Sentiment: neutral
